## Import all the libraries

In [5]:
import os
import numpy as np
from utils.readers import InHospitalMortalityReader

from utils.preprocessing import Discretizer, Normalizer

In [6]:
from utils import utils
import imp
import os

In [176]:
data_path='data'
test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'test'),
                          listfile=os.path.join(data_path, 'test1_listfile.csv'),period_length=48.0)

['10011_episode1_timeseries.csv', '1\n']
10011_episode1_timeseries.csv 1

['10026_episode1_timeseries.csv', '0\n']


In [170]:
## normalizer
cont_channels=[2, 3, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
normalizer = Normalizer(fields=cont_channels) 
normalizer_state = 'utils/ihm_ts1.0.input_str:previous.start_time:zero.normalizer'
normalizer.load_params(normalizer_state)

In [171]:
## discretizer
discretizer = Discretizer(timestep=1.0,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

config_path~~~~~~~in preprocessing.py /home/ec2-user/SageMaker/Deployment/MIMIC-LSTM-Mortality-deployment/utils/discretizer_config.json


In [177]:
test_data_gen = utils.BatchGen(reader=test_reader,
                           discretizer=discretizer,
                           normalizer=normalizer,
                           partition='none',
                           batch_size=16,
                           steps=None,  
                           shuffle=False,
                           return_names=True)

## Prepare Model

In [12]:
arg_dict= {'batch_norm': False, 'batch_size': 16, 'beta_1': 0.9, 'data': '/home/ec2-user/SageMaker/mimic3-benchmarks/data/in-hospital-mortality', 'depth': 2, 'dim': 16, 'dropout': 0.3, 'epochs': 60, 'imputation': 'previous', 'l1': 0, 'l2': 0, 'load_state': '', 'lr': 0.001, 'mode': 'train', 'network': 'mimic3models/keras_models/lstm.py', 'normalizer_state': None, 'optimizer': 'adam', 'output_dir': 'mimic3models/in_hospital_mortality', 'prefix': '', 'rec_dropout': 0.0, 'save_every': 1, 'size_coef': 4.0, 'small_part': False, 'target_repl_coef': 0.0, 'timestep': 1.0, 'verbose': 2, 'header': ['Capillary refill rate->0.0', 'Capillary refill rate->1.0', 'Diastolic blood pressure', 'Fraction inspired oxygen', 'Glascow coma scale eye opening->To Pain', 'Glascow coma scale eye opening->3 To speech', 'Glascow coma scale eye opening->1 No Response', 'Glascow coma scale eye opening->4 Spontaneously', 'Glascow coma scale eye opening->None', 'Glascow coma scale eye opening->To Speech', 'Glascow coma scale eye opening->Spontaneously', 'Glascow coma scale eye opening->2 To pain', 'Glascow coma scale motor response->1 No Response', 'Glascow coma scale motor response->3 Abnorm flexion', 'Glascow coma scale motor response->Abnormal extension', 'Glascow coma scale motor response->No response', 'Glascow coma scale motor response->4 Flex-withdraws', 'Glascow coma scale motor response->Localizes Pain', 'Glascow coma scale motor response->Flex-withdraws', 'Glascow coma scale motor response->Obeys Commands', 'Glascow coma scale motor response->Abnormal Flexion', 'Glascow coma scale motor response->6 Obeys Commands', 'Glascow coma scale motor response->5 Localizes Pain', 'Glascow coma scale motor response->2 Abnorm extensn', 'Glascow coma scale total->11', 'Glascow coma scale total->10', 'Glascow coma scale total->13', 'Glascow coma scale total->12', 'Glascow coma scale total->15', 'Glascow coma scale total->14', 'Glascow coma scale total->3', 'Glascow coma scale total->5', 'Glascow coma scale total->4', 'Glascow coma scale total->7', 'Glascow coma scale total->6', 'Glascow coma scale total->9', 'Glascow coma scale total->8', 'Glascow coma scale verbal response->1 No Response', 'Glascow coma scale verbal response->No Response', 'Glascow coma scale verbal response->Confused', 'Glascow coma scale verbal response->Inappropriate Words', 'Glascow coma scale verbal response->Oriented', 'Glascow coma scale verbal response->No Response-ETT', 'Glascow coma scale verbal response->5 Oriented', 'Glascow coma scale verbal response->Incomprehensible sounds', 'Glascow coma scale verbal response->1.0 ET/Trach', 'Glascow coma scale verbal response->4 Confused', 'Glascow coma scale verbal response->2 Incomp sounds', 'Glascow coma scale verbal response->3 Inapprop words', 'Glucose', 'Heart Rate', 'Height', 'Mean blood pressure', 'Oxygen saturation', 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH', 'mask->Capillary refill rate', 'mask->Diastolic blood pressure', 'mask->Fraction inspired oxygen', 'mask->Glascow coma scale eye opening', 'mask->Glascow coma scale motor response', 'mask->Glascow coma scale total', 'mask->Glascow coma scale verbal response', 'mask->Glucose', 'mask->Heart Rate', 'mask->Height', 'mask->Mean blood pressure', 'mask->Oxygen saturation', 'mask->Respiratory rate', 'mask->Systolic blood pressure', 'mask->Temperature', 'mask->Weight', 'mask->pH'], 'task': 'ihm', 'target_repl': False}

In [14]:
network="source/lstm.py"
model_module = imp.load_source(os.path.basename(network), network)
model = model_module.Network(**arg_dict)

Using TensorFlow backend.



==> not used params in network class: dict_keys(['batch_size', 'beta_1', 'data', 'epochs', 'imputation', 'l1', 'l2', 'load_state', 'lr', 'mode', 'network', 'normalizer_state', 'optimizer', 'output_dir', 'prefix', 'save_every', 'size_coef', 'small_part', 'target_repl_coef', 'timestep', 'verbose', 'header'])
final activation is: sigmoid
------------------------------------




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
optimizer_config = {'class_name': 'adam', 'config': {'lr': 0.001, 'beta_1': 0.9}}

loss_function = 'binary_crossentropy'
model.compile(optimizer=optimizer_config,
              loss=loss_function)


In [92]:
model.load_weights("utils/k_lstm.n16.d0.3.dep2.bs16.ts1.0.epoch60.test0.27911064881661624.state")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               (None, None, 76)          0         
_________________________________________________________________
masking_1 (Masking)          (None, None, 76)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 16)          5440      
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 7,569
Trainable params: 7,569
Non-trainable params: 0
_________________________________________________________________


374

In [178]:
names = []
ts = []
labels = []
predictions = []
preds=[]
for i in range(test_data_gen.steps):
            print("\tdone {}/{}".format(i, test_data_gen.steps), end='\r')

            ret = test_data_gen.next(return_y_true=True)
            (x, y_processed, y) = ret["data"]
            cur_names = np.array(ret["names"])#.repeat(x[0].shape[1], axis=-1)
            cur_ts = ret["ts"]
            for single_ts in list(cur_ts):
                ts = ts +[single_ts] 

            pred = model.predict_on_batch(x)
            pred = np.array(pred)[:, 0]
            print(pred)
            
            
            # predictions
            predictions=predictions+list(pred)
            
            #true values
            labels += list(y)
            names += list(cur_names)
            ts += list(cur_ts)

chunk_size is :  18
[0.8027336  0.02925941 0.23275769 0.3569501  0.7338785  0.0310286
 0.23003948 0.03422737 0.6440723  0.28503567 0.01060119 0.5275506
 0.17951354 0.9368766  0.21848345 0.01922765]
[0.01438757 0.74359965]


In [160]:
good_indices=[i for i, j in enumerate(predictions) if j >=0.5]

In [161]:
good_indices

[0,
 1,
 2,
 3,
 11,
 13,
 16,
 19,
 25,
 30,
 33,
 35,
 37,
 43,
 50,
 51,
 58,
 59,
 62,
 63,
 78,
 79,
 80,
 85,
 86,
 90,
 103,
 105,
 108,
 109,
 110,
 122,
 127,
 131,
 132,
 139,
 145,
 147,
 155,
 156,
 157,
 160,
 162,
 163,
 168,
 169,
 170,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 197,
 199,
 202,
 204,
 205,
 207,
 211,
 216,
 220,
 221,
 223,
 229,
 231,
 233,
 237,
 239,
 241,
 242,
 244,
 245,
 249,
 250,
 252,
 257,
 261,
 264,
 265,
 270,
 272,
 274,
 276,
 285,
 299,
 304,
 306,
 308,
 309,
 313,
 319,
 321,
 323,
 328,
 331,
 332,
 334,
 336,
 339,
 340,
 341,
 343,
 344,
 346,
 353,
 356,
 357,
 362,
 365,
 369,
 370,
 371,
 372]

In [162]:
patient_names = [names[i] for i in good_indices]

In [164]:
import pandas as pd
x=pd.read_csv('data/test_ytrue.csv')

In [165]:
x.stay

0      10011_episode1_timeseries.csv
1      10094_episode2_timeseries.csv
2      10102_episode1_timeseries.csv
3      10301_episode2_timeseries.csv
4      10453_episode1_timeseries.csv
                   ...              
369     9905_episode1_timeseries.csv
370    99184_episode1_timeseries.csv
371    99312_episode1_timeseries.csv
372     9936_episode1_timeseries.csv
373    99899_episode1_timeseries.csv
Name: stay, Length: 374, dtype: object

In [166]:
x_sub=x[x.stay.isin(patient_names)]

In [167]:
x_sub.head()

,stay,y_true
8,10822_episode1_timeseries.csv,1
11,11438_episode2_timeseries.csv,1
13,11555_episode1_timeseries.csv,1
23,12909_episode1_timeseries.csv,1
24,13153_episode1_timeseries.csv,1


In [168]:
x_sub.to_csv('data/pos.csv',index=False)

In [174]:
x1=pd.read_csv('data/test2_listfile.csv')

In [175]:
x1.append(x_sub[:6]).to_csv('data/test1_listfile.csv',index=False)